## Building Algorithm 9.1

In [1]:
using UncNLPTestSet, TRS, LinearAlgebra
nlp = SelectProgram("SROSENBR")
adjdim!(nlp, 200);

┌ Info: Precompiling UncNLPTestSet [ac41c6d5-581c-4fd8-9896-caf0766f302e]
└ @ Base loading.jl:1342
┌ Warning: TODO: Debug BDQRTIC
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/BDQRTIC.jl:73
┌ Warning: TODO: Issue in CUTEst? See comment https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl/blob/main/src/broydn7d.jl#L50 
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/BROYDN7D.jl:109
┌ Warning: TODO: Issue in CUTEst? See https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl/blob/main/src/brybnd.jl
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/BRYBND.jl:110
┌ Warning: TODO: Debug CRAGGLVY
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/CRAGGLVY.jl:75
┌ Warning: TODO: Clean up DIXMAANB (and others) implementation. See comment block
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/DIXMAANB.jl:108
┌ Warning: TODO: Implement DIXON3DQ
└ @ UncNLPTestSet ~/.julia/dev/UncNLPTestSet/src/problems/DIXON3DQ.jl:1
┌ Warning: TOD

In [2]:
BLAS.set_num_threads(4)

function qntrHS(nlp, x₀, Hᵢ, Δₖ = 1.0, b=4, ϵ=10e-6)
	η = 0.1
	Δₘ = 1 # out of thin air 
	n = nlp.n
	xₖ = x₀
	fₖ = obj(nlp, xₖ)
	Sₖ = orth(rand(n, 2b-1))
	gₖ, hₖ, Yₖ = gHS(nlp, xₖ, Sₖ)
	Hₖ = BFGS(Hᵢ, [Sₖ gₖ], [Yₖ hₖ], ϵ)
	Hₖ = Symmetric(Hₖ) # this tage may not be needed

	k = 0
	while norm(gₖ) ≥ ϵ  # confirm norm(gₖ).. this may be computed down below 
		Qₖ = [hₖ gₖ Yₖ]  			

		# trust-region subproblem
		P = Symmetric(Qₖ'*Hₖ*Qₖ)
		b = Qₖ'*Hₖ*gₖ
		C = Symmetric(Qₖ'*Hₖ^2*Qₖ)
		aₖ, _ = trs_small(P, b, Δₖ, C)
		aₖ = vec(aₖ) # update TRS ... it returns a column matirx

		# handle result, i.e. step must satisfy convergence criterion
		q = Qₖ*aₖ 
		pₖ = vec(Hₖ*q)
		xₜ = xₖ + pₖ # t for trial
		fₜ = obj(nlp, xₜ) # t for trial
		ρ = (fₜ - fₖ)/(0.5*aₖ'*P*aₖ + P'*aₖ) # confirm I don't need mₖ(0) in denominator

		# should we adjust trust region radius?
		if ρ < η
			Δₖ = 0.25*Δₖ # should I safe gaurd this to be above a lower bound
		elseif ρ > 0.75 && norm(pₖ) ≈ Δₖ
			Δₖ = min(2Δₖ, Δₘ)
		end

		# Update if we took the step?
		if ρ > η
			xₖ = xₖ .+ pₖ
			fₖ = fₜ
			Sₖ = orth(Yₖ - Sₖ*(Sₖ'Yₖ))
			gₖ, hₖ, Yₖ = gHS(nlp, xₖ, Sₖ)
			Hₖ = BFGS(Hᵢ, [Sₖ gₖ], [Yₖ hₖ], ϵ)
		end
		
		# a safeguard
		k+=1
		if k > 50000
			println("Number of iterations: $k")
			return xₖ, gₖ
		end
	end
	println("Number of iterations: $k")
	return xₖ, gₖ
end

qntrHS (generic function with 4 methods)

In [3]:
H0 = rand(nlp.n, nlp.n)
H0 = Symmetric((H0 + H0'))

xₘ, ∇fxₘ = qntrHS(nlp, nlp.x0, H0);

LoadError: UndefVarError: h not defined

In [9]:
[xₘ ∇fxₘ]

200×2 Matrix{Float64}:
 0.594772     70.1866
 0.0553328   -59.6843
 0.621663     85.3274
 0.0402801   -69.237
 0.598064     82.571
 0.00916035  -69.704
 0.508788     23.9503
 0.136355    -24.5021
 0.506379     26.9283
 0.1186      -27.5639
 0.509073     42.8969
 0.0436715   -43.0967
 0.565938     44.4972
 ⋮           
 0.611838      0.379782
 0.369622     -0.944782
 0.655616     14.8964
 0.370403    -11.8859
 0.601218      6.8042
 0.329853     -6.32197
 0.653947     19.3575
 0.350998    -15.3297
 0.669462     43.4053
 0.28362     -32.9118
 0.573831    -11.9176
 0.37749       9.6416

In [10]:
norm(∇fxₘ)

627.8827612954894

It seems that we didn't take any steps. Things work much better when H0 is the identity

In [11]:
H0 = zeros(nlp.n, nlp.n) + I
H0 = Symmetric(H0)

xₘ, ∇fxₘ = qntrHS(nlp, nlp.x0, H0);

Number of iterations: 14700


In [12]:
[xₘ ∇fxₘ]

200×2 Matrix{Float64}:
 1.0   2.2405e-7
 1.0   5.16938e-7
 1.0  -1.89835e-7
 1.0   7.23467e-7
 1.0  -5.36516e-7
 1.0   8.96486e-7
 1.0  -3.83218e-8
 1.0   6.47878e-7
 1.0  -3.49319e-8
 1.0   6.46186e-7
 1.0  -5.33447e-7
 1.0   8.94938e-7
 1.0  -1.2461e-7
 ⋮    
 1.0  -5.39939e-7
 1.0   8.98168e-7
 1.0   1.17135e-7
 1.0   5.70312e-7
 1.0  -2.83736e-7
 1.0   7.7034e-7
 1.0  -7.59637e-8
 1.0   6.66676e-7
 1.0  -2.22905e-7
 1.0   7.39957e-7
 1.0  -4.58901e-7
 1.0   8.57733e-7

In [13]:
norm(∇fxₘ)

8.293264428176477e-6